In [ ]:
import uproot
import matplotlib.pyplot as plt 
import pandas as pd 
import numpy as np 
import matplotlib as mpl

from util import *
import var
import cut
import data
import hist

import importlib

from pyanalib import panda_helpers

In [ ]:
importlib.reload(var)
importlib.reload(cut)
importlib.reload(data)
importlib.reload(hist)
importlib.reload(panda_helpers)

In [ ]:
mpl.rc('font', size=14)

dosave = False
savedir = "./plots/"

import warnings

warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)

In [ ]:
filedir = "/icarus/data/users/gputnam/DP2022P/reco/"

datafile = filedir + "majorityH4_blinded_costrk.df"
# datafile = filedir + "mcnu_costrk.df"

In [ ]:
df = pd.read_hdf(datafile, key="costrk")

In [ ]:
df = df[(df.pfp.parent == -1)]

In [ ]:
df

In [ ]:
yz_ybin = np.linspace(-180, 130, 32)[:-1:2]
yz_ylos = yz_ybin[:-1]
yz_yhis = yz_ybin[1:]
yz_ys = (yz_ylos + yz_yhis) / 2.

yz_zbin = np.linspace(-900, 900, 181)[1:-1:2]
yz_zlos = yz_zbin[:-1]
yz_zhis = yz_zbin[1:]
yz_zs = (yz_zlos + yz_zhis) / 2.

In [ ]:
when = (df.pfp.trk.dir.y < 0) & (df.pfp.parent == -1) & InFV(df.pfp.trk.start, 0, 20, 0, 0)

In [ ]:
cathodeE = (-358.49 + -61.94) / 2
cathodeW = -cathodeE

start_tpcEE = (df.pfp.trk.start.x <  cathodeE) & (df.pfp.trk.producer == 0)
start_tpcEW = (df.pfp.trk.start.x >= cathodeE) & (df.pfp.trk.producer == 0)

start_tpcWE = (df.pfp.trk.start.x <  cathodeW) & (df.pfp.trk.producer == 1)
start_tpcWW = (df.pfp.trk.start.x >= cathodeW) & (df.pfp.trk.producer == 1)

end_tpcEE = (df.pfp.trk.end.x <  cathodeE) & (df.pfp.trk.producer == 0)
end_tpcEW = (df.pfp.trk.end.x >= cathodeE) & (df.pfp.trk.producer == 0)

end_tpcWE = (df.pfp.trk.end.x <  cathodeW) & (df.pfp.trk.producer == 1)
end_tpcWW = (df.pfp.trk.end.x >= cathodeW) & (df.pfp.trk.producer == 1)

tpcEE = start_tpcEE & end_tpcEW
tpcEW = start_tpcEW & end_tpcEE

tpcWE = start_tpcWE & end_tpcWW
tpcWW = start_tpcWW & end_tpcWE

In [ ]:
Nintersect = np.zeros((yz_zlos.size, yz_ylos.size))

minZ = np.minimum(df.pfp.trk.start.z, df.pfp.trk.end.z)
maxZ = np.maximum(df.pfp.trk.start.z, df.pfp.trk.end.z)

dz = df.pfp.trk.end.z - df.pfp.trk.start.z
a = (df.pfp.trk.end.y - df.pfp.trk.start.y) / dz
b = df.pfp.trk.start.y - a * df.pfp.trk.start.z

for i, (zlo, zhi) in enumerate(zip(yz_zlos, yz_zhis)):
    for j, (ylo, yhi) in enumerate(zip(yz_ylos, yz_yhis)):
        rec_maxZ = np.minimum(maxZ, zhi)
        rec_minZ = np.maximum(minZ, zlo)
        
        rec_Y1 = a * rec_minZ + b
        rec_Y2 = a * rec_maxZ + b
        
        rec_minY = np.minimum(rec_Y1, rec_Y2)
        rec_maxY = np.maximum(rec_Y1, rec_Y2)
        
        rec_maxY = np.minimum(rec_maxY, yhi)
        rec_minY = np.maximum(rec_minY, ylo)
        
        Nintersect[i,j] = np.sum(((rec_maxZ >= rec_minZ) & (rec_maxY >= rec_minY))[when & tpcEE])

In [ ]:
# Number of downward-going cosmic tracks that start in each location
plt.figure(0, figsize=(20, 4))

N,_,_,_ = plt.hist2d(df.pfp.trk.start.z[when & tpcEE], df.pfp.trk.start.y[when & tpcEE], bins=[yz_zbin, yz_ybin])

plt.colorbar()


In [ ]:
bins = np.linspace(0, 0.15, 16)
_ = plt.hist((N/Nintersect).flatten(), bins=bins)

In [ ]:
# Number of downward-going cosmic tracks that start in each location as a fraction of all the tracks that have
# gone through that location

plt.figure(0, figsize=(20, 4))

plt.imshow(N.T / Nintersect.T, origin="lower", 
           extent=[yz_zbin[0], yz_zbin[-1], yz_ybin[0], yz_ybin[-1]],
          vmin=0, vmax=0.1)
plt.colorbar()

In [ ]:
when = (df.pfp.trk.dir.y < 0) & (df.pfp.parent == -1) & (df.pfp.id == 0) & tpcEE

fiducial_start = InFV(df.pfp.trk.start, 40, inx=20, iny=40, inzfront=40)

whenFV = when & fiducial_start
whennotFV = when & ~fiducial_start

In [ ]:
# Tracks that have a fiducial start point as a function of track angle - thxz

thxz = np.arctan2(df.pfp.trk.dir.x, df.pfp.trk.dir.z)
bins = np.linspace(0, np.pi, 16)

Nall,_,_ = plt.hist(thxz[when], bins=bins, label="All Tracks")
_ = plt.hist(thxz[whennotFV], bins=bins, histtype="step", linewidth=2, label="Throughgoing")
Ncut,_,_ = plt.hist(thxz[whenFV], bins=bins, histtype="step", linewidth=2, label="Cut")

In [ ]:
# Tracks that have a fiducial start point as a function of track angle -- thxz

plt.plot((bins[:-1] + bins[1:]) / 2., Ncut/Nall)

In [ ]:
# Tracks that have a fiducial start point as a function of track angle -- thyz

thyz = np.arctan2(df.pfp.trk.dir.y, df.pfp.trk.dir.z)
bins = np.linspace(-np.pi, 0, 16)

Nall,_,_ = plt.hist(thyz[when], bins=bins, label="All Tracks")
_ = plt.hist(thyz[whennotFV], bins=bins, histtype="step", linewidth=2, label="Throughgoing")
Ncut,_,_ = plt.hist(thyz[whenFV], bins=bins, histtype="step", linewidth=2, label="Cut")

In [ ]:
# Tracks that have a fiducial start point as a function of track angle -- thyz

plt.plot((bins[:-1] + bins[1:]) / 2., Ncut/Nall)